 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#VGG16-model" data-toc-modified-id="VGG16-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>VGG16 model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Embedding-of-COCO-images" data-toc-modified-id="Embedding-of-COCO-images-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Embedding of COCO images</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#LSTM-+-VGG" data-toc-modified-id="LSTM-+-VGG-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LSTM + VGG</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Index-answers" data-toc-modified-id="Index-answers-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Index answers</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-text-data" data-toc-modified-id="Prepare-text-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Prepare text data</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-the-embedding-layer" data-toc-modified-id="Prepare-the-embedding-layer-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Prepare the embedding layer</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-image-data-I" data-toc-modified-id="Prepare-image-data-I-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Prepare image data I</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-image-data-II-(pass-through-VGG)" data-toc-modified-id="Prepare-image-data-II-(pass-through-VGG)-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Prepare image data II (pass through VGG)</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Sanity-check" data-toc-modified-id="Sanity-check-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Sanity check</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-definition" data-toc-modified-id="Model-definition-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Model definition</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-I-(224x224x3-input)" data-toc-modified-id="Model-I-(224x224x3-input)-2.7.1"><span class="toc-item-num">2.7.1&nbsp;&nbsp;</span>Model I (224x224x3 input)</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-II-(4096-input-from-VGG)" data-toc-modified-id="Model-II-(4096-input-from-VGG)-2.7.2"><span class="toc-item-num">2.7.2&nbsp;&nbsp;</span>Model II (4096 input from VGG)</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Training" data-toc-modified-id="Training-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Predicting-on-test-data-(val2014)" data-toc-modified-id="Predicting-on-test-data-(val2014)-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Predicting on test data (val2014)</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Preparing-test-data" data-toc-modified-id="Preparing-test-data-2.9.1"><span class="toc-item-num">2.9.1&nbsp;&nbsp;</span>Preparing test data</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# Add the path of the main directory
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import keras

from keras.layers import Dense, Dropout, LSTM, multiply, concatenate
from sklearn.model_selection import ShuffleSplit
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.layers import Input
from keras.models import Model

from evaluate import Evaluate

Using TensorFlow backend.


In [3]:
from vqa_api.PythonHelperTools.vqaTools.vqa import VQA

In [4]:
%load_ext autoreload

%autoreload 2

# VGG16 model

In [ ]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tools import img_dir, img_file

In [ ]:
# VGG16 model
vgg_model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', 
                               input_tensor=None, input_shape=None, 
                               pooling=None, classes=1000)

In [ ]:
# Model layers 
layers = vgg_model.layers
for i in range(len(layers)):
    print "--> Layer {}".format(i+1), "\n"
    print layers[i].get_config(), "\n"
    

The fc7 layer is the layer 22, which name is fc2. 

In [ ]:
# fc7 layer
fc7_layer = layers[21]
print fc7_layer.get_config()
fc7_layer_name = fc7_layer.get_config()["name"]
print fc7_layer_name

In [ ]:
# Input shape and output shape of the fc7 layer
print "Input shape:", fc7_layer.input_shape
print "Ouput shape:", fc7_layer.output_shape

In [ ]:
# Weights of the fc7 layer
# Weights shape
print "Weights shape:", fc7_layer.get_weights()[0].shape
# Bias shape
print "Bias shape:", fc7_layer.get_weights()[1].shape

We can also get a Layer by its name:

In [ ]:
# fc7 layer
fc7_layer = vgg_model.get_layer(fc7_layer_name)
fc7_layer.get_config()

## Embedding of COCO images 

In [ ]:
dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"
dataType = "mscoco"

In [ ]:
image_ids = [(9, "train2014"), (25, "train2014")]
# List to store the arrays for images
imgs = []
for image_id in image_ids:
    # Resize the images as VGG inputs
    img = image.load_img(os.path.join(img_dir(dataDir, dataType, 
        image_id[1]), img_file(image_id[1], image_id[0])), target_size=(224, 224)) 
    img = image.img_to_array(img)
    imgs.append(img)
imgs = np.stack(imgs)
# Preprocess the images corresponding to VGG
imgs = preprocess_input(imgs)   

In [ ]:
img = img.reshape(1, 224, 224, 3)

In [ ]:
img.shape

In [ ]:
# "Submodel" of VGG until the fc7 layer
vgg_model_fc7 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

In [ ]:
# Images embedding using the fc7 layer of VGG16
vgg_model_fc7.predict(img).flatten().shape

# LSTM + VGG

In [5]:
from lstm_vgg import LSTMVGG

In [6]:
# dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"
dataDir = "/home/ubuntu/data/vqa"

In [7]:
text_vision = LSTMVGG(dataDir)

--> train2014
loading VQA annotations and questions into memory...
0:00:11.495946
creating index...
index created!


---> val2014
loading VQA annotations and questions into memory...
0:00:04.737271
creating index...
index created!


## Index answers

In [8]:
text_vision.get_top_answers_train()

Getting the top 1000 answers from the training set ...
Mapping each top answer to an index between 0 and 999 in self.idx_to_answer_dic and self.answer_to_idx_dic ...


In [9]:
text_vision.get_most_common_answer_train()

Getting the ground truth answer for each question/image ...


In [10]:
text_vision.reduce_answers_train()

Keeping only every question/image with an answer among the top 1000 answers of the training set ...


In [11]:
text_vision.encode_answers_train()

Encoding train answers ...


In [12]:
text_vision.train_answers_categorical.shape

(214477, 1000)

## Prepare text data 

In [13]:
# Tokenize questions using the vocabulary of the training questions
text_vision.tokenize_questions_train()

Fitting the tokenizer on the training questions ...
Embedding the train questions ...
Padding the sequences to a maximum length of 22 ...


In [14]:
# Number of questions in the train set
len(text_vision.train_sequences)

214477

## Prepare the embedding layer

In [15]:
text_vision.create_embedding_matrix()

In [16]:
text_vision.embedding_matrix.shape

(12494, 300)

## Prepare image data I

In [19]:
# text_vision.process_images_train(n=300)

In [20]:
# text_vision.train_images.shape

## Prepare image data II (pass through VGG)

In [17]:
%time text_vision.process_encode_images_train()

Images have already been encoded, opening them ...
CPU times: user 1.34 s, sys: 836 ms, total: 2.17 s
Wall time: 36.5 s


In [18]:
text_vision.train_images.shape

(214477, 4096)

## Sanity check  

Let's check that the different data correspond (question, answer and image):

In [ ]:
i = 0

In [ ]:
text_vision.train_questions[i]

In [ ]:
text_vision.train_answers[i]

In [ ]:
# plt.imshow(text_vision.train_images[i])
# plt.show()

## Model definition

In [19]:
# Maximum length for the input questions
input_length = 22

# Vocabulary size from the training set
V = text_vision.vocabulary_size_train
input_dim = V + 1

# Dimension of the vectors (from the paper)
embedding_dim = 300

# The output from the embedding layer is equal to input_length x embedding_dim

### Model I (224x224x3 input)

In [24]:
# # VGG16 model
# vgg_model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', 
#                                input_tensor=None, input_shape=None, 
#                                pooling=None, classes=1000)

In [25]:
# # "Submodel" of VGG until the fc7 layer
# vgg_model_fc7 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

In [26]:
# # Let"s get a tensor with the output of our vision model
# image_input = Input(shape=(224, 224, 3))
# # Freeze the vision model weights
# vgg_model_fc7.trainable = False
# fc7 = vgg_model_fc7(image_input)

# # Turn the 4096 embedding from the fc7 layer to a 1024 embedding in order to 
# # match the questions embedding
# encoded_image = Dense(1024, activation="tanh")(fc7)

# # Langage model
# question_input = Input(shape=(22,), dtype="int32")
# embedded_question = Embedding(input_dim=input_dim, output_dim=embedding_dim,
#                               weights=[text_vision.embedding_matrix],
#                               input_length=input_length,
#                               trainable=False
#                              )(question_input)
# encoded_question = LSTM(1024)(embedded_question)

# # Point-wise multiplication of the outputs from the vision model and the 
# # langage model
# merge = multiply([encoded_question, encoded_image])

# # Hidden layers with 0.5 dropouts
# merge = Dropout(0.5)(merge)
# hidden1 = Dense(1000, activation="tanh")(merge)
# hidden1 = Dropout(0.5)(hidden1)
# hidden2 = Dense(1000, activation="tanh")(hidden1)

# # Final softmax layer
# output = Dense(1000, activation="softmax")(hidden2)

# # This is our final model:
# vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# # Summarize layers
# print(vqa_model.summary())
# Plot graph
# plot_model(model, to_file="vision_model.png")

###  Model II (4096 input from VGG)

In [20]:
# Let"s get a tensor with the output of our vision model
image_input = Input(shape=(4096,))

# Turn the 4096 embedding from the fc7 layer to a 1024 embedding in order to 
# match the questions embedding
# encoded_image = image_input
encoded_image = Dense(1024, activation="tanh")(image_input)

# Langage model
question_input = Input(shape=(22,), dtype="int32")
embedded_question = Embedding(input_dim=input_dim, output_dim=embedding_dim,
                              weights=[text_vision.embedding_matrix],
                              input_length=input_length)(question_input)
encoded_question = LSTM(1024)(embedded_question)

# Point-wise multiplication of the outputs from the vision model and the 
# langage model
# merge = concatenate([encoded_question, encoded_image])
merge = multiply([encoded_question, encoded_image])

# Hidden layers with 0.5 dropouts
merge = Dropout(0.5)(merge)
hidden1 = Dense(1000, activation="tanh")(merge)
hidden1 = Dropout(0.5)(hidden1)
hidden2 = Dense(1000, activation="tanh")(hidden1)

# Final softmax layer
output = Dense(1000, activation="softmax")(hidden2)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# Summarize layers
print(vqa_model.summary())
# Plot graph
# plot_model(model, to_file="vision_model.png")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 22, 300)      3748200     input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1024)         5427200     embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (D

## Training 

In [21]:
# Delete variables 
text_vision.clear_variables()

In [29]:
vqa_model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
              metrics=["accuracy"])

In [30]:
# Training set to split
x = [text_vision.train_images, text_vision.train_sequences]
y = text_vision.train_answers_categorical
print(y.shape)

(214477, 1000)


In [ ]:
# Split training set into train set (80%=171580) and validation set
x_train = [x[0][:171580], x[1][:171580]]
x_val = [x[0][171580:], x[1][171580:]]
y_train = y[:171580]
y_val = y[171580:]

In [ ]:
# # Split training set into train set (80%) and validation set (20%)
# rs = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)
# rs.get_n_splits(x[0])
# train_indices = list(rs.split(x[0]))[0][0] 
# val_indices = list(rs.split(x[0]))[0][1]

# x_train = [x[0][train_indices], x[1][train_indices]]
# x_val = [x[0][val_indices], x[1][val_indices]]
# y_train = y[train_indices]
# y_val = y[val_indices]

In [ ]:
# checkpoint
model_name = "lstm-vgg-0.8train"
model_dir = os.path.join(text_vision.dataDir, "Model", "LSTM_VGG")
filepath = os.path.join(model_dir, "_".join((model_name, "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"))) 
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=0, save_best_only=True, mode="max", period=10)
callbacks_list = [checkpoint]

In [ ]:
# Fit the models
history = vqa_model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=100, batch_size=128, callbacks=callbacks_list)

In [ ]:
# Summarize history for accuracy
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(os.path.join(model_dir, "model_accuracy_train_val.jpg"))
plt.show()

In [ ]:
# Summarize history for loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(os.path.join(model_dir, "model_loss_train_val.jpg"))
plt.show()

In [ ]:
# Save the model 
model_suffix = "0.8train_30_epochs"
vqa_model.save(os.path.join(text_vision.dataDir, "Model", "lstm_vgg_{}.h5".format(model_suffix)))

In [ ]:
# Load the model
from keras.models import load_model
model_name = "lstm-vgg-0.8train"
model_dir = os.path.join(text_vision.dataDir, "Model", "LSTM_VGG")
model_path = os.path.join(model_dir, "_".join((model_name, "weights-improvement-90-0.51.hdf5"))) 
vqa_model = load_model(model_path)

## Training on the whole train 

In [31]:
# checkpoint
model_name = "lstm-vgg-train"
model_dir = os.path.join(text_vision.dataDir, "Model", "LSTM_VGG")
filepath = os.path.join(model_dir, "_".join((model_name, "weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"))) 
checkpoint = ModelCheckpoint(filepath, monitor="acc", verbose=0, save_best_only=True, mode="max", period=10)
callbacks_list = [checkpoint]

In [32]:
# Fit the models
history = vqa_model.fit(x, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
214477/214477 [==============================] - 227s 1ms/step - loss: 2.9957 - acc: 0.3279
Epoch 2/50
214477/214477 [==============================] - 226s 1ms/step - loss: 2.2181 - acc: 0.4083
Epoch 3/50
214477/214477 [==============================] - 227s 1ms/step - loss: 2.0630 - acc: 0.4533
Epoch 4/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.9916 - acc: 0.4753
Epoch 5/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.9496 - acc: 0.4876
Epoch 6/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.9197 - acc: 0.4986
Epoch 7/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.8959 - acc: 0.5074
Epoch 8/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.8782 - acc: 0.5145
Epoch 9/50
214477/214477 [==============================] - 227s 1ms/step - loss: 1.8535 - acc: 0.5249
Epoch 10/50
214477/214477 [==============================] - 227s 1ms/ste

## Predicting on test data (val2014)

In [49]:
# Load the model
from keras.models import load_model
model_name = "lstm-vgg-train"
# model_name = "lstm_vgg_train"
model_dir = os.path.join(text_vision.dataDir, "Model", "LSTM_VGG")
model_path = os.path.join(model_dir, "_".join((model_name, "weights-improvement-30-0.61.hdf5"))) 
# model_path = os.path.join(os.path.join(text_vision.dataDir, "Model"), "_".join((model_name, "50_epochs.h5"))) 
vqa_model = load_model(model_path)

### Preparing test data 

In [23]:
# Tokenize questions using the vocabulary of the training questions
text_vision.tokenize_questions_test()

Embedding the test questions ...
Padding the sequences to a maximum length of 22 ...


In [24]:
# Number of questions in the train set
len(text_vision.test_sequences)

121512

In [25]:
# Encode images
%time text_vision.process_encode_images_test()

Images have already been encoded, opening them ...
CPU times: user 896 ms, sys: 524 ms, total: 1.42 s
Wall time: 20.2 s


In [26]:
text_vision.test_images.shape

(121512, 4096)

In [27]:
# Test input to the "LSTM + CNN" 
x_test = [text_vision.test_images, 
         text_vision.test_sequences]

In [50]:
predictions = vqa_model.predict(x_test)

In [51]:
res = text_vision.predictions_to_dic(predictions, text_vision.test_questions_ids)

--> Saving the results


# Evaluation 

In [52]:
# Compute the accuracies for the validation set
eval_ = Evaluate(text_vision.vqa_test, text_vision.results_file, text_vision.quesFile_test, text_vision.dataDir, 
                 text_vision.dataType, text_vision.dataSubTypeTest)
eval_.compute_accuracies()

Loading and preparing results...     
DONE (t=0.61s)
creating index...
index created!
computing accuracy
Finshed Percent: [####################] 99% Done computing accuracy


Overall Accuracy: 50.64%
